In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)
#year = 2023
refund = 102_000   # 112,167.31
dividend = 435_000
expense  = 992_000

today = date.today()
today

datetime.date(2023, 10, 11)

### Restart and Run All Cells

### Record selection for transactions

In [2]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
ORDER BY sells.date DESC, name'''
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
ORDER BY sells.date DESC, name


In [3]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,JMART,2023,9,2023,9,24.10,22.60,1.50,"2,000","48,200.00","45,200.00","3,000.00",6.64,"2,793.14",DTD
1,JMART,2023,9,2023,9,24.70,23.50,1.20,"1,200","29,640.00","28,200.00","1,440.00",5.11,"1,311.90",DTD
2,JMART,2023,8,2023,3,24.60,23.40,1.20,"1,200","29,520.00","28,080.00","1,440.00",5.13,"1,312.41",DTD
3,WHART,2023,8,2022,5,10.60,10.50,0.10,"10,000","106,000.00","105,000.00","1,000.00",0.95,532.66,DOS
4,JMART,2023,8,2023,4,20.60,19.60,1.00,"1,200","24,720.00","23,520.00","1,200.00",5.10,"1,093.15",DTD
5,ICHI,2023,8,2023,6,15.90,14.30,1.60,"1,400","22,260.00","20,020.00","2,240.00",11.19,"2,146.35",DTD
6,TTB,2023,8,2023,7,1.76,1.64,0.12,"15,000","26,400.00","24,600.00","1,800.00",7.32,"1,687.04",DTD
7,JMART,2023,8,2023,7,18.30,17.00,1.30,"3,600","65,880.00","61,200.00","4,680.00",7.65,"4,398.53",DTD
8,JMT,2023,8,2023,6,43.50,39.75,3.75,"1,800","78,300.00","71,550.00","6,750.00",9.43,"6,418.09",DTD
9,ICHI,2023,7,2023,6,15.10,14.30,0.80,100,"1,510.00","1,430.00",80.00,5.59,73.48,DTD


In [4]:
sells_df.groupby(['sell_year','name']).net.sum()

sell_year  name  
2017       AMATA      41,628.30
           BCP        10,733.38
           BEAUTY    132,183.51
           BJC        27,310.83
           CPN        12,673.01
                        ...    
2023       STA      -127,260.28
           TFFIF       2,482.22
           TTB         1,687.04
           WHAIR         656.69
           WHART         532.66
Name: net, Length: 285, dtype: float64

In [6]:
sells_df.groupby(['sell_year'])[['gross','net']].sum()

,gross,net
sell_year,,
2017,"385,700.00","349,110.89"
2018,"285,834.00","223,705.73"
2019,"46,133.00","10,417.22"
2020,"-1,334,580.00","-1,385,545.75"
2021,"627,795.00","536,263.28"
2022,"-149,692.00","-201,413.37"
2023,"-201,150.00","-215,903.35"


In [8]:
sells_df[['gross','net']].sum()

gross   -339,960.00
net     -683,365.35
dtype: float64

In [7]:
total_df = sells_df.groupby(['sell_year'])[['gross','net']].sum()
total_df.nlargest(5, 'gross')[['gross','net']].style.format(format_dict)

,gross,net
sell_year,,
2021,"627,795.00","536,263.28"
2017,"385,700.00","349,110.89"
2018,"285,834.00","223,705.73"
2019,"46,133.00","10,417.22"
2022,"-149,692.00","-201,413.37"


In [7]:
file_name = 'ytd-profit.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
osd_file = osd_path + file_name

sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(csv_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(data_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(osd_file)

In [8]:
output = sells_df.groupby(['sell_month']).net.sum()
output

sell_month
1   -81,338.26
2   -77,891.90
3    -7,863.28
5     3,680.06
6   -75,562.92
7     1,379.68
8    17,588.23
9     4,105.04
Name: net, dtype: float64

In [9]:
data_path = "../data/"
file_name = 'tmp-file-of-profits.csv'
data_file = data_path + file_name
osd_file = osd_path + file_name

output.to_csv(data_file, index=True)
output.to_csv(osd_file, index=True)

In [10]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(-215903.35, 435000, 219096.65)

In [11]:
profit = sells_df.net.sum()
net = profit + dividend - expense
round(profit,0),dividend,expense, round(net,0)

(-215903.0, 435000, 992000, -772903.0)

### Bottom line of up to date transactions

In [12]:
trade = round(profit,0) + dividend
trade

219097.0

In [13]:
total = refund + net
total

-670903.35